# OPE

In [1]:
import pandas as pd
import numpy as np
import random
import concurrent.futures
from sklearn.neighbors import KernelDensity

# variables for the ticket escalation rule
linear = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

ticket_escalation = [1, 1, 1, 2, 3, 5, 8, 10, 11, 12, 12, 12]
# linear: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# logarithmic = [1, 4, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8]
# logistic = [1, 1, 1, 2, 3, 5, 8, 10, 11, 12, 12, 12]

# variables for prize probabilities
prize_money_list = [0, 1, 20, 80]
base_prize_prob_list = [.5, .418, .08, .002]
num_p = 1000 # the number of grid between usual care and usual care plus contingency management

# variables for OPE estimators
seed = 0
gamma = 1.00
M = 1000
reward_name_list = ['stimulant substance-free', 'alcohol-free',
                   'primary substance-free', 'secondary substance-free',
                   'incentives']

processed_df = pd.read_csv('CTN0007_processed.csv')

# line for reproducibility
random.seed(seed)
np.random.seed(seed)

def compute_mean_and_std_of_prize(prize_money_list, prize_prob_list):
    mean_prize = np.sum(np.array(prize_money_list)*np.array(prize_prob_list))
    var_prize = (np.sum((np.array(prize_money_list)**2)*np.array(prize_prob_list))
                 - mean_prize**2)
    std_prize = np.sqrt(var_prize)
    return mean_prize, std_prize

def action_density(num_of_draw_list, prize_prob_list):
    density = 1.0
    for num_of_draw, prize_prob in zip(num_of_draw_list, prize_prob_list):
        density *= (prize_prob**num_of_draw)
        del(num_of_draw, prize_prob)
    return density

def compute_num_of_draw_list(num_of_draw, prize_amount_list, total_prize):
    prize_amounts = np.array(prize_amount_list)
    solutions = []

    # Iterate through all possible combinations of draw distribution
    for combo in np.ndindex(*[num_of_draw + 1] * len(prize_amount_list)):
        if sum(combo) == num_of_draw and np.sum(prize_amounts * np.array(combo)) == total_prize:
            solutions.append(list(combo))

    return solutions

def compute_ipw_estimate(dataframe, reward_name_list,
                         target_escalation_rule,
                         base_prize_prob_list, target_prize_prob_list,
                         gamma,
                         base_prize_amount_list = [0, 1, 20, 80],
                         lottery_result_list = ["NUMBER OF 'GOOD JOBS'", 'NUMBER OF SMALL PRIZES',
                                                'NUMBER OF LARGE PRIZES', 'NUMBER OF JUMBO PRIZES']):
    
    target_discounted_reward_dict = {}
    target_discounted_reward_dict['usubjid'] = []
    for reward_name in reward_name_list:
        target_discounted_reward_dict[reward_name] = []
        del(reward_name)
    
    usubjid_list = np.unique(dataframe['USUBJID'])
    incentive_usubjid_list = np.unique(dataframe[dataframe['SUBJECT RESEARCH GROUP']
                                                 =='INCENTIVE']['USUBJID'])
    control_usubjid_list = np.unique(dataframe[dataframe['SUBJECT RESEARCH GROUP']
                                               =='CONTROL']['USUBJID'])
    for usubjid in usubjid_list:
        current_df = dataframe[dataframe['USUBJID']==usubjid]
        gamma_list = [gamma**i for i in range(len(current_df))]
        reward_dict, is_weight_list = {}, []
        is_weight_list.append(1.0)
        for reward_name in reward_name_list:
            reward_dict[reward_name] = []
            del(reward_name)
        reward_dict['incentives'].append(0)
        for idx in current_df.index:
            num_of_draw_list = np.ndarray.tolist(np.array(dataframe.loc[idx, lottery_result_list]))
            prize = np.sum(np.array(num_of_draw_list)*np.array(base_prize_amount_list))
            if usubjid in incentive_usubjid_list:
                base_action_density = action_density(num_of_draw_list, prize_prob_list=base_prize_prob_list)
            else:
                base_action_density = 1.0
            bonus_draw = (np.array(current_df.loc[idx, ['SECONDARY SUBSTANCE']])[0] == 'NEGATIVE')
            num_of_primary_draw = (np.sum(num_of_draw_list)-2) if ((np.sum(num_of_draw_list)>=2) & bonus_draw) else 0
            target_num_of_primary_draw = 0
            if num_of_primary_draw > 0:
                target_num_of_primary_draw = target_escalation_rule[np.where(np.array(linear)==num_of_primary_draw)[0][0]]
            target_num_of_draw = target_num_of_primary_draw
            if (target_num_of_primary_draw > 0) & bonus_draw:
                target_num_of_draw += 2
            target_num_of_draw_list_list = compute_num_of_draw_list(num_of_draw=target_num_of_draw,
                                                                    prize_amount_list=base_prize_amount_list,
                                                                    total_prize=prize)
            target_action_density = 0
            for target_num_of_draw_list in target_num_of_draw_list_list:
                target_action_density += action_density(target_num_of_draw_list,
                                                        prize_prob_list=target_prize_prob_list)
                del(target_num_of_draw_list)
            
            is_weight = (target_action_density/base_action_density)
            is_weight_list.append(is_weight)
            for reward_name in reward_name_list:
                if reward_name=='stimulant substance-free':
                    reward = 1 if (np.array(current_df.loc[idx, ['STIMULANT SUBSTANCE']])[0] == 'NEGATIVE') else 0
                elif reward_name=='alcohol-free':
                    reward = 1 if (np.array(current_df.loc[idx, ['ALCOHOL']])[0] == 'NEGATIVE') else 0
                elif reward_name=='primary substance-free':
                    reward = 1 if (np.array(current_df.loc[idx, ['PRIMARY SUBSTANCE']])[0] == 'NEGATIVE') else 0
                elif reward_name=='secondary substance-free':
                    reward = 1 if (np.array(current_df.loc[idx, ['SECONDARY SUBSTANCE']])[0] == 'NEGATIVE') else 0
                elif reward_name=='incentives':
                    reward = np.sum(np.array(current_df.loc[idx, lottery_result_list],
                                             dtype='float')
                                    *base_prize_amount_list)
                reward_dict[reward_name].append(reward)
            del(reward)
        del(idx)
        is_weight_list = is_weight_list[:(-1)]
        reward_dict['incentives'] = reward_dict['incentives'][:(-1)]
        for reward_name in reward_name_list:
            target_discounted_reward = np.sum(np.array(gamma_list)*np.array(reward_dict[reward_name])
                                        *np.array(is_weight_list))
            target_discounted_reward_dict[reward_name].append(target_discounted_reward)
        del(reward_name)
        target_discounted_reward_dict['usubjid'].append(usubjid)
    del(usubjid)
    return target_discounted_reward_dict

def compute_bootstrap_results(reward_dict, usubjid_list, seed):
    
    all_usubjid_list = reward_dict['usubjid']
    usubjid_indices = [all_usubjid_list.index(id) for id in usubjid_list]
    
    # compute size of bootstrap results
    local_state = np.random.RandomState(seed)
    total_size = len(usubjid_list)
    
    # bootstrap sample with replacement
    bootstrap_indices = local_state.choice(usubjid_indices, size=total_size, replace=True)
    
    # save bootstrap result
    bootstrap_reward_dict = {}
    for reward_name, reward_list in reward_dict.items():
        if reward_name != 'usubjid':
            bootstrap_reward_list = np.array(reward_list)[bootstrap_indices]
            bootstrap_reward_dict[reward_name] = np.mean(bootstrap_reward_list)
    del(reward_name)    
    return bootstrap_reward_dict

def compute_discounted_reward(policy_name):
    return compute_ipw_estimate(dataframe=processed_df,
                                reward_name_list=reward_name_list,
                                target_escalation_rule=ticket_escalation,
                                base_prize_prob_list=base_prize_prob_list,
                                target_prize_prob_list=prize_prob_dict[policy_name],
                                gamma=gamma)

In [2]:
incentive_usubjid_list = np.unique(processed_df[
    processed_df['SUBJECT RESEARCH GROUP']=='INCENTIVE']['USUBJID'])
control_usubjid_list = np.unique(processed_df[
    processed_df['SUBJECT RESEARCH GROUP']=='CONTROL']['USUBJID'])

In [3]:
prize_prob_dict = {}; mean_dict = {}; std_dict = {}; wr_dict = {}
base_mean, base_std = compute_mean_and_std_of_prize(prize_money_list=prize_money_list,
                                                    prize_prob_list=base_prize_prob_list)
for p in [x/num_p for x in range(0, num_p+1, 1)]:
    current_prob_list = [base_prize_prob_list[i]*(1-p) if i>0
                         else base_prize_prob_list[i]*(1+p) for i in range(len(prize_money_list))]
    current_prob_list[-1] = 1.0 - np.sum(current_prob_list[:-1])
    policy_name = ('$0: %f, $1: %f, $20: %f, $80: %f'
                   % (current_prob_list[0], current_prob_list[1],
                      current_prob_list[2], current_prob_list[3]))
    prize_prob_dict[policy_name] = current_prob_list
    wr_dict[policy_name] = 1.0 - current_prob_list[0]
    del(p, current_prob_list, policy_name)

policy_name_list = list(prize_prob_dict.keys())
for policy_name in policy_name_list:
    prize_mean, prize_std = compute_mean_and_std_of_prize(prize_money_list=prize_money_list,
                                                          prize_prob_list=prize_prob_dict[policy_name])
    mean_dict[policy_name] = prize_mean
    std_dict[policy_name] = prize_std
    del(policy_name)

In [4]:
# Initialize dictionaries to store results
discounted_reward_dict = {}

# First loop: compute discounted_reward for each policy_name
with concurrent.futures.ProcessPoolExecutor() as executor:
    future_to_policy = {executor.submit(compute_discounted_reward, policy_name):
                        policy_name for policy_name in policy_name_list}
    for future in concurrent.futures.as_completed(future_to_policy):
        policy_name = future_to_policy[future]
        try:
            discounted_reward_dict[policy_name] = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (policy_name, exc))

In [5]:
from multiprocessing import Pool

control_bootstrap_results = {policy_name: [] for policy_name in policy_name_list}
incentive_bootstrap_results = {policy_name: [] for policy_name in policy_name_list}

pool = Pool()
for i in range(M):
    for policy_name in policy_name_list:
        control_bootstrap_results[policy_name].append(pool.apply_async(compute_bootstrap_results,
                                                                       [discounted_reward_dict[policy_name],
                                                                        control_usubjid_list,
                                                                        i]))
        incentive_bootstrap_results[policy_name].append(pool.apply_async(compute_bootstrap_results,
                                                                         [discounted_reward_dict[policy_name],
                                                                          incentive_usubjid_list,
                                                                          i]))

# Close the pool and wait for the work to finish
pool.close()
pool.join()

# Now get the results
for policy_name in policy_name_list:
    control_bootstrap_results[policy_name] = [result.get() for result in control_bootstrap_results[policy_name]]
    incentive_bootstrap_results[policy_name] = [result.get() for result in incentive_bootstrap_results[policy_name]]
    
# Create dictionaries to store the results and weighted averages
incentive_IS_bootstrap = {}

# Iterate over the policy names
for policy_name in policy_name_list:
    incentive_IS_estimates = incentive_bootstrap_results[policy_name]
    incentive_IS_bootstrap[policy_name] = [{} for _ in range(len(incentive_IS_estimates))]
    
    # Use a list comprehension to extract all values for a given reward type and compute variance
    for reward_name in reward_name_list:
        # Compute the weighted average for each bootstrap using optimal mix proportion
        for idx, incentive_val in enumerate([IS_estimates[reward_name] for IS_estimates in incentive_IS_estimates]):
            incentive_IS_bootstrap[policy_name][idx][reward_name] = incentive_val

In [6]:
import time

summary_list = []
total_start_time = time.time()

for i in range(M):
    start_time = time.time()
    for reward_name in reward_name_list:
        for policy_name in policy_name_list:
            discounted_reward = incentive_IS_bootstrap[policy_name][i][reward_name]
            summary_list.append([i, gamma, reward_name, policy_name, discounted_reward])
    end_time = time.time()
    elapsed_time = end_time - start_time
    total_elapsed_time = end_time - total_start_time
    eta = total_elapsed_time / (i+1) * (M-i-1)
    print(f'Processed results for i = {i}, Expected remaining time: {eta:.2f} seconds')

summary_df = pd.DataFrame(summary_list, columns=['bootstrap id', 'gamma', 'reward name', 'policy', 'discounted reward'])

Processed results for i = 0, Expected remaining time: 2.31 seconds
Processed results for i = 1, Expected remaining time: 2.12 seconds
Processed results for i = 2, Expected remaining time: 2.04 seconds
Processed results for i = 3, Expected remaining time: 1.95 seconds
Processed results for i = 4, Expected remaining time: 1.95 seconds
Processed results for i = 5, Expected remaining time: 1.89 seconds
Processed results for i = 6, Expected remaining time: 1.89 seconds
Processed results for i = 7, Expected remaining time: 1.87 seconds
Processed results for i = 8, Expected remaining time: 1.94 seconds
Processed results for i = 9, Expected remaining time: 2.01 seconds
Processed results for i = 10, Expected remaining time: 2.03 seconds
Processed results for i = 11, Expected remaining time: 2.06 seconds
Processed results for i = 12, Expected remaining time: 2.07 seconds
Processed results for i = 13, Expected remaining time: 2.12 seconds
Processed results for i = 14, Expected remaining time: 2.1

Processed results for i = 158, Expected remaining time: 1.70 seconds
Processed results for i = 159, Expected remaining time: 1.70 seconds
Processed results for i = 160, Expected remaining time: 1.70 seconds
Processed results for i = 161, Expected remaining time: 1.69 seconds
Processed results for i = 162, Expected remaining time: 1.69 seconds
Processed results for i = 163, Expected remaining time: 1.69 seconds
Processed results for i = 164, Expected remaining time: 1.68 seconds
Processed results for i = 165, Expected remaining time: 1.68 seconds
Processed results for i = 166, Expected remaining time: 1.68 seconds
Processed results for i = 167, Expected remaining time: 1.68 seconds
Processed results for i = 168, Expected remaining time: 1.68 seconds
Processed results for i = 169, Expected remaining time: 1.68 seconds
Processed results for i = 170, Expected remaining time: 1.68 seconds
Processed results for i = 171, Expected remaining time: 1.68 seconds
Processed results for i = 172, Exp

Processed results for i = 361, Expected remaining time: 1.99 seconds
Processed results for i = 362, Expected remaining time: 1.98 seconds
Processed results for i = 363, Expected remaining time: 1.98 seconds
Processed results for i = 364, Expected remaining time: 1.97 seconds
Processed results for i = 365, Expected remaining time: 1.97 seconds
Processed results for i = 366, Expected remaining time: 1.96 seconds
Processed results for i = 367, Expected remaining time: 1.96 seconds
Processed results for i = 368, Expected remaining time: 1.95 seconds
Processed results for i = 369, Expected remaining time: 1.95 seconds
Processed results for i = 370, Expected remaining time: 1.94 seconds
Processed results for i = 371, Expected remaining time: 1.94 seconds
Processed results for i = 372, Expected remaining time: 1.93 seconds
Processed results for i = 373, Expected remaining time: 1.93 seconds
Processed results for i = 374, Expected remaining time: 1.92 seconds
Processed results for i = 375, Exp

Processed results for i = 585, Expected remaining time: 1.85 seconds
Processed results for i = 586, Expected remaining time: 1.84 seconds
Processed results for i = 587, Expected remaining time: 1.84 seconds
Processed results for i = 588, Expected remaining time: 1.83 seconds
Processed results for i = 589, Expected remaining time: 1.83 seconds
Processed results for i = 590, Expected remaining time: 1.82 seconds
Processed results for i = 591, Expected remaining time: 1.81 seconds
Processed results for i = 592, Expected remaining time: 1.81 seconds
Processed results for i = 593, Expected remaining time: 1.80 seconds
Processed results for i = 594, Expected remaining time: 1.79 seconds
Processed results for i = 595, Expected remaining time: 1.79 seconds
Processed results for i = 596, Expected remaining time: 1.78 seconds
Processed results for i = 597, Expected remaining time: 1.78 seconds
Processed results for i = 598, Expected remaining time: 1.77 seconds
Processed results for i = 599, Exp

Processed results for i = 767, Expected remaining time: 1.11 seconds
Processed results for i = 768, Expected remaining time: 1.11 seconds
Processed results for i = 769, Expected remaining time: 1.10 seconds
Processed results for i = 770, Expected remaining time: 1.09 seconds
Processed results for i = 771, Expected remaining time: 1.09 seconds
Processed results for i = 772, Expected remaining time: 1.08 seconds
Processed results for i = 773, Expected remaining time: 1.08 seconds
Processed results for i = 774, Expected remaining time: 1.07 seconds
Processed results for i = 775, Expected remaining time: 1.07 seconds
Processed results for i = 776, Expected remaining time: 1.06 seconds
Processed results for i = 777, Expected remaining time: 1.05 seconds
Processed results for i = 778, Expected remaining time: 1.05 seconds
Processed results for i = 779, Expected remaining time: 1.04 seconds
Processed results for i = 780, Expected remaining time: 1.04 seconds
Processed results for i = 781, Exp

Processed results for i = 930, Expected remaining time: 0.36 seconds
Processed results for i = 931, Expected remaining time: 0.36 seconds
Processed results for i = 932, Expected remaining time: 0.35 seconds
Processed results for i = 933, Expected remaining time: 0.34 seconds
Processed results for i = 934, Expected remaining time: 0.34 seconds
Processed results for i = 935, Expected remaining time: 0.33 seconds
Processed results for i = 936, Expected remaining time: 0.33 seconds
Processed results for i = 937, Expected remaining time: 0.32 seconds
Processed results for i = 938, Expected remaining time: 0.32 seconds
Processed results for i = 939, Expected remaining time: 0.31 seconds
Processed results for i = 940, Expected remaining time: 0.31 seconds
Processed results for i = 941, Expected remaining time: 0.30 seconds
Processed results for i = 942, Expected remaining time: 0.30 seconds
Processed results for i = 943, Expected remaining time: 0.29 seconds
Processed results for i = 944, Exp

In [7]:
import pandas as pd

df_for_plot = summary_df.copy()  # Work on a copy of the data

# Compute summary statistics
agg_data_mean = df_for_plot.groupby(['reward name', 'policy'])['discounted reward'].mean().unstack().fillna(0)
agg_data_2_5 = df_for_plot.groupby(['reward name', 'policy'])['discounted reward'].quantile(0.025).unstack().fillna(0)
agg_data_97_5 = df_for_plot.groupby(['reward name', 'policy'])['discounted reward'].quantile(0.975).unstack().fillna(0)
agg_data_std = df_for_plot.groupby(['reward name', 'policy'])['discounted reward'].std().unstack().fillna(0)


In [8]:
# In this code, we evaluated the logistic escalation rule
df_for_plot.to_csv('logistic.csv', index=False)